In [1]:
%pwd

'c:\\Users\\ranja\\end-to-end-care-generative-AI\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\ranja\\end-to-end-care-generative-AI'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [9]:
extracted_data = load_pdf_file('./Data')


In [10]:
print(os.getcwd())


c:\Users\ranja\end-to-end-care-generative-AI


In [11]:
#extracted_data

In [11]:
# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents (extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("length of text chunks",len(text_chunks))

length of text chunks 5699


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings


In [14]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [16]:
embeddings=download_hugging_face_embeddings()

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()


In [18]:
query_result=embeddings.embed_query("hello world")
print("length",len(query_result))

length 384


In [191]:
#query_result

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
PINECONE_API_KEY=os.environ.get('PINCONE_API_KEY')
COHERA_API_KEY=os.environ.get('OPENAI_API_KEY')


In [21]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_3pivjw_DyTaZc32XRucj9sc6JJTbxEbrReY96AtEbbpjkLKrsjJHvjbaqmR5nerYZQR9Gh")


In [22]:
#from pinecone.grpc import PineconeGRPC as Pinecone
#from pinecone import ServerlessSpec

#pc = Pinecone(api_key="pcsk_fNHUg_9r1qEX2iirXAZhSAQwXdqaU976t2KUj8Kz5xiDZKXAdtKXtHm1zWrf8FnZeYcJi")
#pc.create_index(
 #   name="carebot",
  #  dimension=384,
   # metric="cosine",
    #spec=ServerlessSpec(
     #   cloud="aws",
      #  region="us-east-1"
    #)
#)

In [23]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Define your Pinecone API key and index name
pc = Pinecone(api_key="pcsk_3pivjw_DyTaZc32XRucj9sc6JJTbxEbrReY96AtEbbpjkLKrsjJHvjbaqmR5nerYZQR9Gh")  # Replace with your real key
index_name = "carebot"

# Check if index exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print(f"Index '{index_name}' already exists.")

# Use the index
index = pc.Index(index_name)


In [24]:
import os

PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")
COHERE_API_KEY=os.environ.get("COHERE_API_KEY")

In [25]:
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY
os.environ["COHERE_API_KEY"]=COHERE_API_KEY

In [26]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents =text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [27]:
print(os.environ.get("PINECONE_API_KEY"))
print(os.environ.get("COHERE_API_KEY")  # Should print your key
)

pcsk_fNHUg_9r1qEX2iirXAZhSAQwXdqaU976t2KUj8Kz5xiDZKXAdtKXtHm1zWrf8FnZeYcJi
YgRZwrEJjTlHQvuSvWff6TtJK62anJ3hZBpPIfLw


In [28]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings

)

In [29]:
docsearch

In [30]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [31]:
retrieved_docs=retriever.invoke("what is acne?")

In [32]:
retrieved_docs

[Document(id='c4983baf-9c71-48eb-84be-87a8d7255a9c', metadata={'author': 'Clifford', 'creationdate': '2004-12-28T15:38:25-05:00', 'creator': 'PyPDF', 'enhanced': 'By PDF Enhancer 2.5/Win', 'moddate': '2005-05-04T13:53:15-06:00', 'page': 4.0, 'page_label': '5', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\medical_book.pdf', 'spdf': '1096', 'total_pages': 599.0}, page_content='A\nAbscess\nAcidophilus\nAcne\nAconite\nAcupressure\nAcupuncture\nAdemetionine\nAdie’s pupil\nAfrican pygeum\nAgastache\nAging\nAIDS\nAlcoholism\nAlexander technique\nAlfalfa\nAlisma\nAllergies\nAllium cepa\nAloe\nAlpha-hydroxy\nAlzheimer’s disease\nAmino acids\nAndrographis\nAndrostenedione\nAnemarrhena\nAnemia\nAngelica root\nAngina\nAnise\nAnkylosing spondylitis\nAnorexia nervosa\nAnthroposophical medicine\nAntioxidants\nAnxiety\nBates method\nBayberry\nBedsores\nBedwetting\nBee pollen\nBehavioral optometry\nBehavioral therapy'),
 Document(id='501c856a-4b8e-4c1e-ac23-6ec00e48cdb1', metadata={'author'

In [33]:

from langchain_community.llms import Cohere

llm=Cohere(temperature=0.4,max_tokens=500)

C:\Users\ranja\AppData\Local\Temp\ipykernel_13268\795543653.py:3: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm=Cohere(temperature=0.4,max_tokens=500)


In [34]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer,say that you"
    "don't know. Use three sentences maximum and keep the \n\n{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [35]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [36]:
response=rag_chain.invoke({"input":"What is fever?"})
print(response["answer"])

 Fever is an elevation of the body's temperature to at least 100 degrees Fahrenheit (37.8 degrees Celsius). Fevers are part of the body's response to an infection as pathogens do not thrive at higher temperatures. The immune system's white blood cells work best in a warm environment. 


In [37]:
response=rag_chain.invoke({"input": "What is stats?"})
print (response["answer"])


 Sorry, the provided context does not mention anything related to stats or statistics. I don't have any information on what "stats" refers to in this context. 

Could you please provide more context or clarify what you would like to know? Then, I can answer your question to the best of my ability. 


In [38]:
response=rag_chain.invoke({"input": "What is acromegaly?"})
print (response["answer"])

 I don't know but I can provide some information on Acromegaly. 

Acromegaly is a hormonal condition that produces abnormal growth of the hands, feet, and jaws, as well as the gums and soft tissues of the mouth. It most commonly presents in adulthood and can be a result of a benign tumor on the pituitary gland that produces an excess amount of growth hormone (GH). 

In addition to abnormal growth, acromegaly can also cause other complications such as arthritis, fatigue, and vision problems. Treatment often involves surgery or radiation to remove the tumor, or medical options that can suppress the excess GH and reduce the complications. 

I hope this helps provide some useful information on acromegaly. Let me know if there's anything else I can help with. 
